In [58]:
import os
from langchain.utils import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import OpenAIEmbeddings
from langchain.utils import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from dotenv import load_dotenv
from operator import itemgetter
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


load_dotenv()
os.environ['OPENAPI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

######### Data #########
mysql_uri = 'mysql+mysqlconnector://root@localhost:3306/nikprod'
model = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
db = SQLDatabase.from_uri(mysql_uri )
chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())
retriver = chroma_db.as_retriever()

def get_schema(_):
    return db.get_table_info()
def run_query(query):
    return db.run(query)



In [46]:
with open('data/topic.txt',encoding='utf8') as f:
    lines = f.readlines()
template = """
"""
prompt = ChatPromptTemplate.from_template(template)
prompt



ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are good at classifying a question.\n    Given the user question below, classify the question as either begin about "Database" or "Chat".\n    <If the question is like these sample than its a \'Database\' question>\n    SAMPLE:\n    Can you list all the products available in the database?"\n    "What is the price of the product with ID 123?"\n    "How many units of the product named \'Shower Head Model X\' are in stock?"\n    "Show me the details of the product with the SKU \'ABCD1234\'."\n    "Which products were added to the database in the last month?"]\n    "How many different types of shower heads do we have?"\n    "Can you fetch the product names and prices for all products in the \'Premium\' category?"\n    "What are the dimensions of the \'Eco Shower System\'?"\n    "How many reviews does the product \'Rain Shower XL\' have?"\n 

In [59]:
system="""
You are good at classifying a question.
    Given the user question below, classify the question as either begin about "Database" or "Chat".
    <If the question is like these sample than its a 'Database' question>
    SAMPLE:
    Can you list all the products available in the database?"
    "What is the price of the product with ID 123?"
    "How many units of the product named 'Shower Head Model X' are in stock?"
    "Show me the details of the product with the SKU 'ABCD1234'."
    "Which products were added to the database in the last month?"]
    "How many different types of shower heads do we have?"
    "Can you fetch the product names and prices for all products in the 'Premium' category?"
    "What are the dimensions of the 'Eco Shower System'?"
    "How many reviews does the product 'Rain Shower XL' have?"
    "What is the average rating of the product 'Eco-Friendly Shower'?"
    "How many products are currently out of stock?"
    "List all the products that are on sale."
    "What are the specifications of the product with ID 456?"
    "Which products have a discount greater than 20%?"
    "What are the top 5 best-selling products?"
    "How many products are there in the 'Luxury' category?"
    "Fetch the supplier details for the product 'High-Pressure Shower Head'."
    "What is the reorder level for the product 'Compact Shower System'?"
    "List all the products with a 5-star rating."
    "Show me all products that were discontinued last year."
    "What is the manufacturing date of the product with SKU 'EFG5678'?"
    "Can you provide a summary of the latest product additions to the database?"
    "What are the shipping details for the product 'Dual Shower System'?"
    "List all products that have received customer complaints."
    "What is the return policy for the product 'Adjustable Shower Arm'?"
    "How many products are supplied by 'Nikles Manufacturing'?"
    "Fetch the inventory levels for all products in the 'Economy' range."
    "What are the dimensions and weight of the 'Ultra-Thin Shower Head'?"
    "How many units of 'Deluxe Shower Kit' were sold last month?"
    "What is the customer feedback for the product 'Thermostatic Shower Mixer'?"
    "List all products that require special installation services."
    "What is the lead time for the product 'Handheld Shower Wand'?"
    "Can you show the sales trends for the product 'Luxury Rain Shower' over the past year?"
    <If the question is about warrenty or Nikles company products or other, then its a 'Chat' topic>
    
    Classification:
"""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
           system,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model | StrOutputParser


In [ ]:
res = chain.invoke({"messages":["what is nikles?"]})


In [67]:

classification_template = PromptTemplate.from_template(
    """
    Given the user question below, classify the question as either begin about "Database" or "Chat".
    <If the question is like these sample than its a 'Database' question>
    SAMPLE:
    Can you list all the products available in the database?"
    "What is the price of the product with ID 123?"
    "How many units of the product named 'Shower Head Model X' are in stock?"
    "Show me the details of the product with the SKU 'ABCD1234'."
    "Which products were added to the database in the last month?"]
    "How many different types of shower heads do we have?"
    "Can you fetch the product names and prices for all products in the 'Premium' category?"
    "What are the dimensions of the 'Eco Shower System'?"
    "How many reviews does the product 'Rain Shower XL' have?"
    "What is the average rating of the product 'Eco-Friendly Shower'?"
    "How many products are currently out of stock?"
    "List all the products that are on sale."
    "What are the specifications of the product with ID 456?"
    "Which products have a discount greater than 20%?"
    "What are the top 5 best-selling products?"
    "How many products are there in the 'Luxury' category?"
    "Fetch the supplier details for the product 'High-Pressure Shower Head'."
    "What is the reorder level for the product 'Compact Shower System'?"
    "List all the products with a 5-star rating."
    "Show me all products that were discontinued last year."
    "What is the manufacturing date of the product with SKU 'EFG5678'?"
    "Can you provide a summary of the latest product additions to the database?"
    "What are the shipping details for the product 'Dual Shower System'?"
    "List all products that have received customer complaints."
    "What is the return policy for the product 'Adjustable Shower Arm'?"
    "How many products are supplied by 'Nikles Manufacturing'?"
    "Fetch the inventory levels for all products in the 'Economy' range."
    "What are the dimensions and weight of the 'Ultra-Thin Shower Head'?"
    "How many units of 'Deluxe Shower Kit' were sold last month?"
    "What is the customer feedback for the product 'Thermostatic Shower Mixer'?"
    "List all products that require special installation services."
    "What is the lead time for the product 'Handheld Shower Wand'?"
    "Can you show the sales trends for the product 'Luxury Rain Shower' over the past year?"
    <If the question is about warrenty or Nikles company products or other, then its a 'Chat' topic>
    
    <question>
    {question}
    Classification:
    </question>

    Classification:"""
)

classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [69]:
classification_chain.invoke({"question":"how Good Nikles"})

'Chat'

In [57]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


KeyError: "Input to ChatPromptTemplate is missing variables {'question'}.  Expected: ['question'] Received: ['query']"